# 1. Load Library & Data

In [69]:
import numpy as np
import matplotlib.pyplot as plt

### Data load & preprocessing 

In [70]:
from dataset.mnist import load_mnist

(train_raw_img, train_y), (test_raw_img, test_y) = load_mnist(flatten=False, normalize=False)

# reshape 
train_X = train_raw_img.reshape(len(train_raw_img.squeeze()), -1)
test_X = test_raw_img.reshape(len(test_raw_img.squeeze()), -1)

In [71]:
# preprocessing : normalization(set 0~1)
train_X = train_X.astype('float')/255
test_X = test_X.astype('float')/255

In [72]:
print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


# 2. Model

In [73]:
def information_gain(y, y_left, y_right):
    def entropy(y):
        class_labels = np.unique(y)
        entropy = 0
        for label in class_labels:
            p = np.sum(y == label) / len(y)
            entropy += -p * np.log2(p)
        return entropy

    entropy_before = entropy(y)
    entropy_left = entropy(y_left)
    entropy_right = entropy(y_right)
    
    entropy_after = (len(y_left) / len(y)) * entropy_left + (len(y_right) / len(y)) * entropy_right
    return entropy_before - entropy_after

In [115]:
def make_tree(X, y, max_depth, depth=0) :
    if depth == max_depth or len(np.unique(y)) == 1 : # stopping conditions
        return {'class' : np.argmax(np.bincount(y))} # return class
    
    best_feature, best_threshold = find_best_split(X, y)
    left_mask = X[:, best_feature] <= best_threshold
    right_mask = X[:, best_feature] > best_threshold
    left_node = make_tree(X[left_mask], y[left_mask], max_depth, depth + 1, )
    right_node = make_tree(X[right_mask], y[right_mask], max_depth, depth + 1)
    
    return {'feature': best_feature, 'threshold': best_threshold, 'left': left_node, 'right': right_node}

def find_best_split(X, y):
    best_gain = -1
    best_feature = None
    best_threshold = None

    for feature_idx in range(X.shape[1]):
        feature_values = X[:, feature_idx]
        thresholds = np.unique(feature_values)

        for threshold in thresholds:
            y_left = y[feature_values <= threshold]
            y_right = y[feature_values > threshold]

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            gain = information_gain(y, y_left, y_right)

            if gain > best_gain:
                best_gain = gain
                best_feature = feature_idx
                best_threshold = threshold

    return best_feature, best_threshold

def prediction(x, node) :
    if 'class' in node :
        return node['class']
    feature = node['feature']
    feature = node['feature']
    threshold = node['threshold']

    if x[feature] <= threshold:
        return prediction(x, node['left'])
    else:
        return prediction(x, node['right'])

### train & evaluation

In [34]:
# early stopping
model = make_tree(train_X, train_y, max_depth=0)
pred1 = np.array([prediction(x, model) for x in train_X])
pred2 = np.array([prediction(x, model) for x in test_X])

print('early stopping in train set :', np.mean(pred1==train_y))
print('early stopping in test set :', np.mean(pred2==test_y))

early stopping in train set : 0.11236666666666667
early stopping in test set : 0.1135


#### 1) math depth = 1

In [37]:
# early stopping
model1 = make_tree(train_X, train_y, max_depth=1)
pred1 = np.array([prediction(x, model1) for x in train_X])
pred2 = np.array([prediction(x, model1) for x in test_X])

print('early stopping in train set :', np.mean(pred1==train_y))
print('early stopping in test set :', np.mean(pred2==test_y))

early stopping in train set : 0.20411666666666667
early stopping in test set : 0.206


#### 2) math depth = 2

In [39]:
# early stopping
model2 = make_tree(train_X, train_y, max_depth=2)
pred1 = np.array([prediction(x, model2) for x in train_X])
pred2 = np.array([prediction(x, model2) for x in test_X])

print('early stopping in train set :', np.mean(pred1==train_y))
print('early stopping in test set :', np.mean(pred2==test_y))

early stopping in train set : 0.33665
early stopping in test set : 0.3398


#### 3) math depth = 3

In [41]:
# early stopping
model3 = make_tree(train_X, train_y, max_depth=3)
pred1 = np.array([prediction(x, model3) for x in train_X])
pred2 = np.array([prediction(x, model3) for x in test_X])

print('early stopping in train set :', np.mean(pred1==train_y))
print('early stopping in test set :', np.mean(pred2==test_y))

early stopping in train set : 0.4881333333333333
early stopping in test set : 0.4918


#### 4) math depth = 4

In [ ]:
# early stopping
model4 = make_tree(train_X, train_y, max_depth=4)
pred1 = np.array([prediction(x, model4) for x in train_X])
pred2 = np.array([prediction(x, model4) for x in test_X])

print('early stopping in train set :', np.mean(pred1==train_y))
print('early stopping in test set :', np.mean(pred2==test_y))

#### 5) math depth = 5

In [43]:
# early stopping
model5 = make_tree(train_X, train_y, max_depth=5)
pred1 = np.array([prediction(x, model5) for x in train_X])
pred2 = np.array([prediction(x, model5) for x in test_X])

print('early stopping in train set :', np.mean(pred1==train_y))
print('early stopping in test set :', np.mean(pred2==test_y))

early stopping in train set : 0.6933333333333334
early stopping in test set : 0.6995


# [Bagging]

In [40]:
def bagging_with_decision_trees(X, y, num_trees, max_depth) :
    
    models = []
    
    for _ in range(num_trees) :
        bootstrap_idx = np.random.choice(len(X), size=len(X), replace=True)
        bootstrap_X, bootstrap_y = X[bootstrap_idx], y[bootstrap_idx]
        model = make_tree(bootstrap_X, bootstrap_y, max_depth)
        models.append(model)
        
    return models

In [47]:
def predict_bagging(models, X):
    predictions = []

    for model in models:
        model_predictions = []
        for i in range(len(X)):
            pred = prediction(X[i], model)
            model_predictions.append(pred)
        predictions.append(model_predictions)

    predictions = np.array(predictions)
    majority_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
    
    return majority_vote

In [48]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 0)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.1135

In [55]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 1)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.206

In [50]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 2)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.3365

In [51]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 3)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.5145

In [52]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 4)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.6676

In [53]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 5)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.7594

In [54]:
models = bagging_with_decision_trees(train_X, train_y, num_trees = 10, max_depth = 5)

preds = predict_bagging(models, test_X)
acc = np.mean(preds==test_y)
acc

0.7694

### Random forests

In [84]:
def make_tree(X, y, max_depth, depth=0) :
    if depth == max_depth or len(np.unique(y)) == 1 : # stopping conditions
        return {'class' : np.argmax(np.bincount(y))} # return class
    
    best_feature, best_threshold = find_best_split(X, y)
    left_mask = X[:, best_feature] <= best_threshold
    right_mask = X[:, best_feature] > best_threshold
    left_node = make_tree(X[left_mask], y[left_mask], max_depth, depth + 1, )
    right_node = make_tree(X[right_mask], y[right_mask], max_depth, depth + 1)
    
    return {'feature': best_feature, 'threshold': best_threshold, 'left': left_node, 'right': right_node}

def find_best_split(X, y):
    best_gain = -1
    best_feature = None
    best_threshold = None

    for feature_idx in range(X.shape[1]):
        feature_values = X[:, feature_idx]
        thresholds = np.unique(feature_values)

        for threshold in thresholds:
            y_left = y[feature_values <= threshold]
            y_right = y[feature_values > threshold]

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            gain = information_gain(y, y_left, y_right)

            if gain > best_gain:
                best_gain = gain
                best_feature = feature_idx
                best_threshold = threshold

    return best_feature, best_threshold

def prediction(x, node):
    if 'class' in node:
        return node['class']
    feature = node['feature']
    threshold = node['threshold']
    
    if x[0, feature] <= threshold:  # 개별 요소에 대해 비교
        return prediction(x, node['left'])
    else:
        return prediction(x, node['right'])

In [95]:
def random_forests_with_decision_trees(X, y, num_trees, max_depth):
    models = []
    feature_idx = []

    for _ in range(num_trees):
        bootstrap_idx = np.random.choice(len(X), size=len(X), replace=True)
        bootstrap_X, bootstrap_y = X[bootstrap_idx], y[bootstrap_idx]

        num_features = int(np.sqrt(X.shape[1]))
        selected_features = np.random.choice(X.shape[1], size=num_features, replace=False)
        feature_idx.append(selected_features)

        model = make_tree(bootstrap_X[:, selected_features], bootstrap_y, max_depth)
        models.append(model)

    return models, feature_idx

In [96]:
def predict_random_forests(models, X, feature_idx):
    predictions = []

    for model in models:
        model_predictions = []
        for i in range(len(X)):
            pred = prediction(X[i, feature_idx], model)  # Only consider selected features
            model_predictions.append(pred)
        predictions.append(model_predictions)

    predictions = np.array(predictions)
    majority_vote = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)

    return majority_vote

In [97]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 0)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1135

In [99]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 1)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1519

In [110]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 1)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1235

In [114]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 2)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1554

In [101]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 2)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1316

In [104]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 3)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.2132

In [111]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 3)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1887

In [105]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 4)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1618

In [112]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 4)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1218

In [109]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 10, max_depth = 4)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.1355

In [103]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 5)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.3102

In [107]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 5)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.187

In [108]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 10, max_depth = 5)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.2884

In [113]:
models, feature_idx = random_forests_with_decision_trees(train_X, train_y, num_trees = 6, max_depth = 6)

preds = predict_random_forests(models, test_X, feature_idx)
acc = np.mean(preds==test_y)
acc

0.2384